# Encoder-Decoder Model

Here, we will experiment with an Encoder-Decoder model for this sequence-to-sequence problem. We will primarily be following [this tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

Outline:\
I. Imports and Data\
II. Model Creation\
III. Model Training\
IV. Model Evaluation

## I. Imports and Data

In [379]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, Normalization, BatchNormalization, TimeDistributed

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

In [380]:
# Load data from the HDF5 file
with h5py.File('enc_dec_preprocessed_data.h5', 'r') as hf:
    x = hf['X'][:]
    y = hf['y'][:]
    decoder_inputs = hf["decoder_inputs"][:]

In [381]:
x[0].shape

(21, 35)

In [382]:
y[0].shape

(35,)

In [383]:
NUM_PLAYS = 21
NUM_FEATURES = 35
hidden_size = 128
EPOCHS = 5
BATCH_SIZE = 128
MASK_VAL = -1.1

# Create Decoder Input Data

In [384]:
# decoder_inputs = []
# decoder_outputs = []
# START_TOKEN = np.zeros_like(y[0]) + -1.1  

# for i, y_ex in enumerate(y):
#     if i == 0:
#         decoder_inputs.append(np.expand_dims(START_TOKEN, axis=0))
#     else:
#         decoder_inputs.append(np.expand_dims(y[i-1],axis=0))

# decoder_inputs = np.array(decoder_inputs)
# decoder_inputs = pad_sequences(decoder_inputs, maxlen=21, padding='pre', dtype='float32', value=MASK_VAL)
# decoder_inputs.shape

In [385]:
x_train[0]

array([[-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1

In [386]:
decoder_inputs[0]

array([[-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00,
        -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00, -1.100e+00],
       [-1.100e+00, -1.100e+00, -1.100e+00, -1

In [387]:
y_train[0]

array([6.500e+01, 1.670e+02, 1.067e+03, 2.867e+03, 0.000e+00, 4.000e+00,
       0.000e+00, 1.000e+00, 4.000e+00, 0.000e+00, 1.000e+00, 9.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 3.000e+00, 3.000e+00, 3.000e+00,
       0.000e+00, 0.000e+00, 3.000e+00, 0.000e+00, 3.000e+00, 3.000e+00,
       0.000e+00, 3.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00])

# Train-Test Split

In [388]:
x_train, x_test, decoder_inputs_train, decoder_inputs_test, y_train, y_test = train_test_split(x, decoder_inputs, y, test_size=0.2, random_state=1819)

In [389]:
x_train.shape, x_test.shape

((212103, 21, 35), (53026, 21, 35))

In [390]:
y_train.shape, y_test.shape

((212103, 35), (53026, 35))

In [391]:
decoder_inputs_train.shape, decoder_inputs_test.shape

((212103, 21, 35), (53026, 21, 35))

## Model Creation

### Encoder

In [392]:
encoder_inputs = Input((NUM_PLAYS, NUM_FEATURES))
encoder_inputs = Masking(mask_value=MASK_VAL)(encoder_inputs)
encoder_inputs = BatchNormalization()(encoder_inputs)

encoder = LSTM(hidden_size, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

### Decoder

In [393]:
decoder_inputs = Input((NUM_PLAYS, NUM_FEATURES))
decoder_inputs = Masking(mask_value=MASK_VAL)(decoder_inputs)
decoder_inputs = BatchNormalization()(decoder_inputs)
# decoder_inputs = norm_layer(decoder_inputs)
decoder_lstm = LSTM(hidden_size, return_sequences=False, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(NUM_FEATURES, activation="relu")
decoder_outputs = decoder_dense(decoder_outputs)


In [394]:
print(decoder_outputs.shape)

(None, 35)


### Full Model

In [395]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [396]:
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy", "MSE"])

In [397]:
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_61      │ (None, 21, 35)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_40        │ (None, 21, 35)    │          0 │ input_layer_61[1… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_388CL… │ (None, 21, 35)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_40 (Any)        │ (None, 21)        │          0 │ not_equal_40[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_396CL… │ (None, 21, 35)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_59 (LSTM)      │ [(None, 128),     │     83,968 │ keras_tensor_388… │
│                     │ (None, 128),      │            │ any_40[1][0]      │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_60 (LSTM)      │ [(None, 128),     │     83,968 │ keras_tensor_396… │
│                     │ (None, 128),      │            │ lstm_59[1][1],    │
│                     │ (None, 128)]      │            │ lstm_59[1][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 35)        │      4,515 │ lstm_60[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 172,451 (673.64 KB)

 Trainable params: 172,451 (673.64 KB)

 Non-trainable params: 0 (0.00 B)

In [398]:
print(f"x_train shape: {x_train.shape}")
print(f"decoder_inputs_train shape: {decoder_inputs_train.shape}")
print(f"y_train shape: {y_train.shape}")

x_train shape: (212103, 21, 35)
decoder_inputs_train shape: (212103, 21, 35)
y_train shape: (212103, 35)


## Model Training

In [399]:
history = model.fit(x=[x_train, decoder_inputs_train], y=y_train, 
                    epochs=EPOCHS, batch_size=BATCH_SIZE, 
                    validation_data=[[x_test, decoder_inputs_test], y_test])

Epoch 1/5


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_388CLONE', 'keras_tensor_396CLONE']. Received: the structure of inputs=('*', '*')
  warnings.warn(


KeyError: "Exception encountered when calling Functional.call().\n\n\x1b[1m14839596336\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=('tf.Tensor(shape=(None, 21, 35), dtype=float32)', 'tf.Tensor(shape=(None, 21, 35), dtype=float32)')\n  • training=True\n  • mask=('None', 'None')"

# Model Evaluation

In [ ]:
results = model.evaluate([x_test, decoder_inputs_test], y_test, batch_size=128)

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


# Plot Model

In [ ]:
tf.keras.utils.plot_model(model=model,
         show_shapes=True, to_file='./enc-dec.png')